In [1]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf
import numpy as np
import os

In [2]:
original_size_img = (1664, 1128)
half_size_img = (1177, 798)
five_percent_size_img = (372, 252)

In [3]:
! ls ../data/localdata/chest_xray/val/NORMAL

NORMAL2-IM-1427-0001.jpeg NORMAL2-IM-1436-0001.jpeg NORMAL2-IM-1440-0001.jpeg
NORMAL2-IM-1430-0001.jpeg NORMAL2-IM-1437-0001.jpeg NORMAL2-IM-1442-0001.jpeg
NORMAL2-IM-1431-0001.jpeg NORMAL2-IM-1438-0001.jpeg


In [4]:
# Directory path
train_data_dir = '../data/localdata/chest_xray/train'
test_data_dir = '../data/localdata/chest_xray/test'

# Get all the data in the directory data/localdata/chest_xray/test (624 images), and reshape them
generator = ImageDataGenerator(rescale=1/255, validation_split=0.25)

train_set = generator.flow_from_directory(
        train_data_dir, target_size=(372, 252), batch_size=32, color_mode='grayscale', subset='training', class_mode='binary')

val_set =  test_set = generator.flow_from_directory(
        train_data_dir, target_size=(372, 252), batch_size=32, color_mode='grayscale', subset='validation', class_mode='binary' )


test_set = generator.flow_from_directory(
        test_data_dir, target_size=(372, 252), batch_size=32, color_mode='grayscale', class_mode='binary')

# Create the datasets
train_images, train_labels = next(train_set)
test_images, test_labels = next(test_set)

Found 3913 images belonging to 2 classes.
Found 1303 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## First Complicated Model

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(372, 252, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05)))

Metal device set to: Apple M1


2023-09-11 16:05:57.934019: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-11 16:05:57.934169: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [7]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision
model.compile(optimizer='Adam',
              loss='BinaryCrossentropy',
              metrics='Accuracy')

In [8]:
model.fit(train_set, epochs=10, batch_size=200, validation_data=val_set, workers=5)

Epoch 1/10


2023-09-11 16:05:58.350701: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-11 16:05:58.662837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


123/123 [==============================] - ETA: 0s - loss: 1.9792 - Accuracy: 0.8400

2023-09-11 16:06:58.391511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


123/123 [==============================] - 68s 546ms/step - loss: 1.9792 - Accuracy: 0.8400 - val_loss: 0.8219 - val_Accuracy: 0.8488
Epoch 2/10
123/123 [==============================] - 68s 553ms/step - loss: 0.4212 - Accuracy: 0.9494 - val_loss: 0.4190 - val_Accuracy: 0.8749
Epoch 3/10
123/123 [==============================] - 68s 552ms/step - loss: 0.2106 - Accuracy: 0.9578 - val_loss: 0.2440 - val_Accuracy: 0.9317
Epoch 4/10
123/123 [==============================] - 64s 516ms/step - loss: 0.1699 - Accuracy: 0.9550 - val_loss: 0.2141 - val_Accuracy: 0.9325
Epoch 5/10
123/123 [==============================] - 63s 506ms/step - loss: 0.1296 - Accuracy: 0.9668 - val_loss: 0.1454 - val_Accuracy: 0.9616
Epoch 6/10
123/123 [==============================] - 65s 528ms/step - loss: 0.1137 - Accuracy: 0.9693 - val_loss: 0.1883 - val_Accuracy: 0.9271
Epoch 7/10
123/123 [==============================] - 64s 516ms/step - loss: 0.1080 - Accuracy: 0.9714 - val_loss: 0.1778 - val_Accuracy: 0.9

In [10]:
model.evaluate(test_images, test_labels)

1/1 [==============================] - 1s 637ms/step - loss: 0.5046 - Accuracy: 0.8750


[0.50458163022995, 0.875]

## Simple Model (Dense Layers)

## Simple Model (CNN)